# Day 24

In [16]:
from dataclasses import dataclass
from enum import Enum
from typing import Callable


class Operation(Enum):
    AND = 1
    OR = 2
    XOR = 3

    def __str__(self) -> str:
        return self.name

    def __repr__(self) -> str:
        return self.name


@dataclass
class Gate:
    left: str
    right: str
    output_wire: str
    operation: Operation

    def perform_operation(self, wires: dict[str, int]) -> int:
        left, right = wires.get(self.left), wires.get(self.right)
        if left is None:
            return -1

        if right is None:
            return -2

        if self.operation == Operation.AND:
            return left & right
        elif self.operation == Operation.OR:
            return left | right
        elif self.operation == Operation.XOR:
            return left ^ right
        else:
            raise ValueError(f"Invalid operation: {self.operation}")


def find_value(value: str, wires: dict[str, int], gates: list[Gate]) -> int:
    if value.isdigit():
        return int(value)

    for g in gates:
        if g.output_wire == value:
            result = g.perform_operation(wires)
            if result == -1:
                left_value = find_value(g.left, wires, gates)
                wires[g.left] = left_value
                return find_value(value, wires, gates)

            elif result == -2:
                right_value = find_value(g.right, wires, gates)
                wires[g.right] = right_value
                return find_value(value, wires, gates)

            return result

    return -1


def gates_wires(file_name: str) -> tuple[dict[str, int], list[Gate]]:
    wires, g = open(file_name).read().split("\n\n")
    wires = {w.split(":")[0]: int(w.split(":")[1]) for w in wires.strip().split("\n")}
    gates = []
    for gate in [wire for wire in g.strip().split("\n")]:
        left, operation, right, _, output_wire = gate.split(" ")
        gates.append(Gate(left, right, output_wire, Operation[operation.upper()]))

    return wires, gates


def get_number(
    filt: Callable[[str], bool], wires: dict[str, int], gates: list[Gate]
) -> int:
    for gate in gates:
        result = gate.perform_operation(wires)
        if result == -1 or result == -2:
            result = find_value(gate.output_wire, wires, gates)
            if result == -1 or result == -2:
                assert False, f"Should never happen: {result}"

        wires[gate.output_wire] = result

    num = 0

    for key in sorted(filter(filt, wires.keys()), reverse=True):
        num = (num << 1) | wires[key]

    return num


def part_1() -> int:
    wires, gates = gates_wires("Input/InputDay24P1.txt")
    return get_number(lambda x: x.startswith("z"), wires, gates)


print(part_1())

55920211035878


In [21]:
def parse_input(input_data):
    lines = input_data.strip().split('\n')
    
    wires = {}
    gates = []
    
    for line in lines:
        if '->' in line:
            parts = line.split(' -> ')
            operation = parts[0].strip()
            output_wire = parts[1].strip()
            gates.append((operation, output_wire))
        else:
            wire, value = line.split(': ')
            wires[wire] = int(value)
    
    return wires, gates

def evaluate_gate(operation, wires):
    for gate_type in ['AND', 'OR', 'XOR']:
        if f' {gate_type} ' in operation:
            parts = operation.split(f' {gate_type} ')
            input1 = parts[0].strip()
            input2 = parts[1].strip()
            
            val1 = wires.get(input1, None)
            val2 = wires.get(input2, None)
            
            if val1 is not None and val2 is not None:
                if gate_type == 'AND':
                    return val1 & val2
                elif gate_type == 'OR':
                    return val1 | val2
                elif gate_type == 'XOR':
                    return val1 ^ val2
    
    raise ValueError(f"Invalid operation: {operation}")

def simulate_circuit(wires, gates):
    evaluated = set()
    
    while True:
        progress_made = False
        
        for operation, output_wire in gates:
            if output_wire not in evaluated:
                try:
                    result = evaluate_gate(operation, wires)
                    wires[output_wire] = result
                    evaluated.add(output_wire)
                    progress_made = True
                except ValueError:
                    continue
        
        if not progress_made:
            break
    
    return wires

def get_z_output_as_decimal(wires):
    z_values = [wires[key] for key in sorted(wires.keys()) if key.startswith('z')]
    binary_str = ''.join(map(str, z_values))
    return int(binary_str, 2)

# Example input
input_data = """
x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1
ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj
"""

with open("Input/InputDay24P1.txt", "r") as f:
    real_input = f.read()


# Parse the input data
initial_wires, gate_operations = parse_input(real_input)

# Simulate the circuit
final_wires = simulate_circuit(initial_wires, gate_operations)

# Get the decimal output from wires starting with 'z'
output_decimal = get_z_output_as_decimal(final_wires)
print(output_decimal)  # Output should be 2024 for this example

28416619574995


# Part 2

In [18]:


def find_output_wire(
    left: str, right: str, operation: Operation, gates: list[Gate]
) -> str | None:
    for g in gates:
        if g.left == left and g.right == right and g.operation == operation:
            return g.output_wire

        if g.left == right and g.right == left and g.operation == operation:
            return g.output_wire

    return None


def full_adder_logic(
    x: str, y: str, c0: str | None, gates: list[Gate], swapped: list
) -> tuple[str | None, str | None]:
    """
    Full Adder Logic:
    A full adder adds three one-bit numbers (X1, Y1, and carry-in C0) and outputs a sum bit (Z1) and a carry-out bit (C1).
    The logic for a full adder is as follows:
    - X1 XOR Y1 -> M1 (intermediate sum)
    - X1 AND Y1 -> N1 (intermediate carry)
    - C0 AND M1 -> R1 (carry for intermediate sum)
    - C0 XOR M1 -> Z1 (final sum)
    - R1 OR N1 -> C1 (final carry)

    Args:
    - x: input wire x
    - y: input wire y
    - c0: input carry
    - gates: list of gates
    - swapped: list of swapped wires

    Returns:
    - z1: final sum
    - c1: final carry

    References:
    - https://www.geeksforgeeks.org/full-adder/
    - https://www.geeksforgeeks.org/carry-look-ahead-adder/
    - https://en.wikipedia.org/wiki/Adder_(electronics)#Full_adder
    """

    # X1 XOR Y1 -> M1 (intermediate sum)
    m1 = find_output_wire(x, y, Operation.XOR, gates)

    # X1 AND Y1 -> N1 (intermediate carry)
    n1 = find_output_wire(x, y, Operation.AND, gates)

    assert m1 is not None, f"m1 is None for {x}, {y}"
    assert n1 is not None, f"n1 is None for {x}, {y}"

    if c0 is not None:
        # C0 AND M1 -> R1 (carry for intermediate sum)
        r1 = find_output_wire(c0, m1, Operation.AND, gates)
        if not r1:
            n1, m1 = m1, n1
            swapped.append(m1)
            swapped.append(n1)
            r1 = find_output_wire(c0, m1, Operation.AND, gates)

        # C0 XOR M1 -> Z1 (final sum)
        z1 = find_output_wire(c0, m1, Operation.XOR, gates)

        if m1 and m1.startswith("z"):
            m1, z1 = z1, m1
            swapped.append(m1)
            swapped.append(z1)

        if n1 and n1.startswith("z"):
            n1, z1 = z1, n1
            swapped.append(n1)
            swapped.append(z1)

        if r1 and r1.startswith("z"):
            r1, z1 = z1, r1
            swapped.append(r1)
            swapped.append(z1)

        assert r1 is not None, f"r1 is None for {c0}, {m1}"
        assert n1 is not None, f"n1 is None for {c0}, {m1}"

        # R1 OR N1 -> C1 (final carry)
        c1 = find_output_wire(r1, n1, Operation.OR, gates)
    else:
        z1 = m1
        c1 = n1

    return z1, c1


def part_2() -> str:
    wires, gates = gates_wires("Input/InputDay24P1.txt")

    # Populate all wires
    get_number(lambda x: x.startswith("z"), wires, gates)

    c0 = None  # carry
    swapped = []  # list of swapped wires

    bits = len([wire for wire in wires if wire.startswith("x")])
    for i in range(bits):
        n = str(i).zfill(2)
        x = f"x{n}"
        y = f"y{n}"

        z1, c1 = full_adder_logic(x, y, c0, gates, swapped)

        if c1 and c1.startswith("z") and c1 != "z45":
            c1, z1 = z1, c1
            swapped.append(c1)
            swapped.append(z1)

        # update carry
        c0 = c1 if c1 else find_output_wire(x, y, Operation.AND, gates)

    return ",".join(sorted(swapped))


print(part_2())

btb,cmv,mwp,rdg,rmj,z17,z23,z30
